In [1]:
# importing all dependencies (just in case)

import gmaps
import numpy as np
import pandas as pd
import requests
import time
from datetime import datetime
from scipy.stats import linregress
from matplotlib import pyplot as plt
from config_Laura import (gkey2)

In [2]:
# Zillow has their datasets divided by how many bedrooms there are: importing all datasets, to then merge them and clean them
zillow_value_1 = pd.read_csv('Resources_Laura/zillow_value_1.csv')
zillow_value_2 = pd.read_csv('Resources_Laura/zillow_value_2.csv')
zillow_value_3 = pd.read_csv('Resources_Laura/zillow_value_3.csv')

In [3]:
# adding a column to show how many bedrooms, based on the dataset, so that it is clear once it is merged.
zillow_value_1['Bdrm'] = "1"
zillow_value_2['Bdrm'] = "2"
zillow_value_3['Bdrm'] = "3"
len(zillow_value_1)+len(zillow_value_2)+len(zillow_value_3)

74047

In [4]:
# merging all datasets together to work out of one clean document
properties = pd.concat([zillow_value_1,zillow_value_2,zillow_value_3])
len(properties)

74047

In [5]:
# filter for california properties only:
properties_ca = properties.loc[properties['State']=="CA"]
len(properties_ca)

4544

In [6]:
# remove columns that have values prior to date 12/31/2013 - we are analyzing the CAGR post 2014.
properties_ca = properties_ca[['RegionName','RegionType','StateName','State','City','Metro','CountyName','1/31/14','2/28/21','3/31/21','Bdrm']]
properties_ca.head()

RegionName RegionType StateName State           City  \
12       94109        Zip        CA    CA  San Francisco   
19       90250        Zip        CA    CA      Hawthorne   
37       94565        Zip        CA    CA      Pittsburg   
41       90046        Zip        CA    CA    Los Angeles   
83       94501        Zip        CA    CA        Alameda   

                             Metro            CountyName   1/31/14  2/28/21  \
12   San Francisco-Oakland-Hayward  San Francisco County  716780.0   824861   
19  Los Angeles-Long Beach-Anaheim    Los Angeles County  382463.0   608374   
37   San Francisco-Oakland-Hayward   Contra Costa County       NaN   192942   
41  Los Angeles-Long Beach-Anaheim    Los Angeles County  452491.0   647755   
83   San Francisco-Oakland-Hayward        Alameda County  348254.0   562250   

    3/31/21 Bdrm  
12   820867    1  
19   614481    1  
37   189534    1  
41   650469    1  
83   566397    1

In [7]:
# dropping nas
properties_ca = properties_ca.dropna()
len(properties_ca)

3923

In [8]:
# calculate annual growth rate since 2014
properties_ca['% Change'] = ((properties_ca['3/31/21'] - properties_ca['1/31/14'])/properties_ca['1/31/14']*100)
properties_ca = pd.DataFrame(properties_ca)
properties_ca = properties_ca.sort_values(by='% Change', ascending=True)
properties_ca = properties_ca.dropna()

In [9]:
#create the list of unique zip codes for the API request (to avoid duplicates)
properties_zip = pd.DataFrame({'RegionName':properties_ca['RegionName'],'County':properties_ca['CountyName']})
properties_zip = properties_zip.drop_duplicates()
len(properties_zip)

1485

In [10]:
# creating lat and long columns for final run
properties_zip['Lat'] = ""
properties_zip['Lng'] = ""

In [11]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey2}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in properties_zip.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    zipcode = row['RegionName']

    # update address key value
    params['address'] = (zipcode)

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    properties_zip.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    properties_zip.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [15]:
properties_zip.head()

ZipCode                County      Lat      Lng
18298    95412         Sonoma County  38.7139 -123.326
16475    95017     Santa Cruz County  37.0919 -122.231
18436    94020      San Mateo County  37.2911 -122.209
20053    94060      San Mateo County  37.2051 -122.333
6759     93110  Santa Barbara County  34.5084 -119.731

In [13]:
result = pd.merge(properties_ca, properties_zip, on="RegionName")
result = result.drop(columns=['County'])
result.head()

RegionName RegionType StateName State       City                   Metro  \
0       95412        Zip        CA    CA  Annapolis              Santa Rosa   
1       95412        Zip        CA    CA  Annapolis              Santa Rosa   
2       95412        Zip        CA    CA  Annapolis              Santa Rosa   
3       95017        Zip        CA    CA  Davenport  Santa Cruz-Watsonville   
4       95017        Zip        CA    CA  Davenport  Santa Cruz-Watsonville   

          CountyName   1/31/14  2/28/21  3/31/21 Bdrm   % Change      Lat  \
0      Sonoma County  721046.0   526204   520369    1 -27.831373  38.7139   
1      Sonoma County  730825.0   653418   645435    2 -11.684056  38.7139   
2      Sonoma County  772083.0   935029   951287    3  23.210458  38.7139   
3  Santa Cruz County  789701.0   609039   612187    1 -22.478634  37.0919   
4  Santa Cruz County  669451.0   826864   847012    2  26.523375  37.0919   

       Lng  
0 -123.326  
1 -123.326  
2 -123.326  
3 -122.231  
4 -122.231

In [17]:
#rename region name to zip code (within the zipcode coordinates dataset), so that it is more clear to the rest of the contributors
properties_zip = properties_zip.rename(columns={'RegionName': 'ZipCode'})
properties_zip.head()

ZipCode                County      Lat      Lng
18298    95412         Sonoma County  38.7139 -123.326
16475    95017     Santa Cruz County  37.0919 -122.231
18436    94020      San Mateo County  37.2911 -122.209
20053    94060      San Mateo County  37.2051 -122.333
6759     93110  Santa Barbara County  34.5084 -119.731

In [18]:
#save files to resources folder
result.to_csv("Resources_Laura/clean_property_value.csv", index = False)
properties_zip.to_csv("Resources_Laura/zipcode_coordinates.csv", index = False)